In [1]:
from enum import auto, IntFlag
from typing import Any
from pydantic import (
    BaseModel,
    EmailStr,
    Field,
    SecretStr,
    ValidationError,
)
#importação das bibliotecas necessárias 

In [ ]:
#Classe Role
#Usa intflag, que permite que os valores sejam automaticamente enumerados e combinados, como na caso do admin. a função auto() define que cada instância recebe um valor que é uma potência de 2
#author = 1, editor = 2, developer = 4, admin = 1+2+4 = 7
class Role(IntFlag):
    Author = auto()
    Editor = auto()
    Developer = auto()
    Admin = Author | Editor | Developer

In [ ]:
#Classe User 
class User(BaseModel):
    #O nome, email, senha e role, serão inseridos como como entrada na instância da classe User.
    #Note que o conteúdo dos campos é definido pelos módulos importados de pydantic.
    #Eles fazem uma "pré-validação" dos dados que serão inseridos.
    name : str = Field(examples= ["Arjan"])
    #O email deve conter a estrutura de um email "abc@def.com", caso contrário haverá um erro na instanciação
    email : EmailStr = Field(
        examples = ["exampleam@arjancodes.com"],
        description = "The email adress of the user",
        frozen = True,
    )
    #A senha será recebida e em caso de visualização, ela será exposta de forma anonimizada 
    password : SecretStr = Field(
        examples= ["Password123"], description="The password of the user" 
    )
    #Como default=None, esse campo é opcional. Mas caso não fosse, receberia os valores de Role, como Role.Author.
    #Na visualização é possível obter com print(User.role) a saída Role.
    # (alguma das trẽs), ou, por exemplo print(int(User.role)) uma saída numérica que corresponde aos valores da intflag
    role: Role = Field(
        default=None,
        description="The role of the user"
    )
    

In [ ]:
#validade recebe um dicionário que contém os valores da classe Usuário e faz a validação deles.
def validate(data: dict[str, Any]) -> None:
    try:
        user = User.model_validate(data)
        print(user)
    except ValidationError as e:
        #No caso de uma entrada inválida, a saída irá conter os erros de forma a possibilitar
        #  de forma simples a correção da falha.
        print("User is invalid")
        for error in e.errors():
            print(error)

In [ ]:
#Nessa função principal a função validade valida 2 exemplos de entradas. A boa entrada é aprovada, enquanto a entrada ruim é rejeitada.
def main() -> None:
    good_data = {
        "name":"Arjan",
        "email":"example@arjancodes.com",
        "password":"Password123",
    }
    bad_data = {
        "email":"<bad data>","password": "<bad data>"
    }

    validate(good_data)
    validate(bad_data)

if __name__ == "__main__":
        main()

name='Arjan' email='example@arjancodes.com' password=SecretStr('**********') role=None
User is invalid
{'type': 'missing', 'loc': ('name',), 'msg': 'Field required', 'input': {'email': '<bad data>', 'password': '<bad data>'}, 'url': 'https://errors.pydantic.dev/2.10/v/missing'}
{'type': 'value_error', 'loc': ('email',), 'msg': 'value is not a valid email address: An email address must have an @-sign.', 'input': '<bad data>', 'ctx': {'reason': 'An email address must have an @-sign.'}}


In [ ]:
#exemplo de validação
usuario_correto = User(name = "Pedro", email = "Pedro@gmail.com", password= "Pedro123", role=1)
validate(usuario_correto)

name='Pedro' email='Pedro@gmail.com' password=SecretStr('**********') role=<Role.Author: 1>


In [ ]:
#exemplo de erro produzido em razão de dados inválidos
usuario_invalido = User(name = "Pedro123", email = "Pedro(@)gmail,com", password= "Pedro123")
validate(usuario_invalido)

ValidationError: 1 validation error for User
email
  value is not a valid email address: The email address contains invalid characters before the @-sign: '('. [type=value_error, input_value='Pedro(@)gmail,com', input_type=str]